In [17]:
import warnings
warnings.filterwarnings('ignore')
import gempy as gp
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import pickle
from gempy.assets import topology as tp
from tqdm import tqdm_notebook
import seaborn as sns
import pandas as pd
from tqdm import tqdm

## Geometric model

In [18]:
surf_model_1 = pd.read_csv(r'C:\Users\Josefine\OneDrive\Dokumente\Masterarbeit\Model_Notebooks\Versuch zwei\Data\test_sur.csv')
x_max=surf_model_1['X'].max()
x_min=surf_model_1['X'].min()
y_max=surf_model_1['Y'].max()
y_min=surf_model_1['Y'].min()
z_max=surf_model_1['Z'].max()
z_min=surf_model_1['Z'].min()-500


In [19]:
geo_model1 = gp.create_model('Model1')
# Importing the data from CSV-files and setting extent and resolution
gp.init_data(geo_model1, [x_min, x_max, y_min, y_max, z_min, z_max], [100,100,50],
             path_o= r'C:\Users\Josefine\OneDrive\Dokumente\Masterarbeit\Model_Notebooks\Versuch zwei\Data\test_or.csv',
             path_i= r'C:\Users\Josefine\OneDrive\Dokumente\Masterarbeit\Model_Notebooks\Versuch zwei\Data\test_sur.csv',
             default_values=False)

gp.map_stack_to_surfaces(geo_model1,
                         {
                         "Fault_1": ('fault_1'), 
                          "Fault_2": ('fault_2'), 
                          "Fault_3": ('fault_3'), 
                          "Fault_4": ('fault_4'),
                          "Fault_5": ('fault_5'), 
                          "Strat_Series":(
                              'UFM', 
                              'UMM',
                              'LFM', 
                              'helvetic',
                              'mesozoic',
                              'basement'
                          ),                                
                         },remove_unused_series=True)

geo_model1.set_is_fault(['Fault_1',
                        'Fault_2', 
                        'Fault_3', 
                        'Fault_4', 
                        'Fault_5',
                      ],change_color=True)
interp_data = gp.set_interpolator(geo_model1,
                                 compile_theano=True,
                                 theano_optimizer='fast_run', gradient=False,
                                 dype='float32')
geo_model1.surfaces.colors.change_colors({
                                        'UFM':'#F2F542' , 
                                        'UMM':'#917E11',
                                         'LFM':'#CCBD6A',
                                          'helvetic': '#6BA0BF',
                                          'mesozoic': '#6BA0BF', 
                                          'basement' : '#6BA0BF'
                                       })
gp.compute_model(geo_model1)

Active grids: ['regular']
Fault colors changed. If you do not like this behavior, set change_color to False.
Setting kriging parameters to their default values.
Compiling theano function...
Level of Optimization:  fast_run
Device:  cpu
Precision:  float64
Number of faults:  5
Compilation Done!
Kriging values: 
                              values
range                  96979.546529
$C_o$              223929343.928571
drift equations  [3, 3, 3, 3, 3, 3]



Lithology ids 
  [11.         11.         10.99996567 ...  7.          7.
  7.        ] 

In [4]:
gp.plot_3d(geo_model1, image=False, show_lith=False, show_data=True, show_results=True, plotter_type='background')


## Uncertainty 

Credits to Elisa Heim and Sofia brisson

In [20]:
import copy
import sys
sys.path.append("./")
import fishdist as fish

In [21]:
def create_vMF_list(geo_model, kappas, datatype = 'all'):
    surfaces_df = geo_model.surfaces.df
    orient_df = geo_model.orientations.df
    vMF_list=[]
    
    if datatype == 'faultsonly':
        faults = list(surfaces_df[surfaces_df['isFault'] == True].index)
        df = orient_df[orient_df.surface.isin(faults)]
        
    elif datatype == 'lithonly':
        notfaults = list(surfaces_df[surfaces_df['isFault'] == False].index)
        df = orient_df[orient_df.series.isin(notfaults)]
        
    elif datatype == 'all':
        df = orient_df
        
    else:
        print('nö.')
    for e, i in df[['G_x', 'G_y', 'G_z']].iterrows():
        a = 0
        vMF_list.append(fish.vMF('vMF_' + str(e), mean=i[['G_x', 'G_y', 'G_z']].values, kappa=kappas[a]))
        a += 1
    return vMF_list

In [22]:
#save the different values to be able to make the model virgin again after each uncertainty loop
surf_x_1 = geo_model1.surface_points.df['X']
surf_y_1 = geo_model1.surface_points.df['Y']
surf_z_1 = geo_model1.surface_points.df['Z']

or_x = geo_model1.orientations.df['X']
or_y = geo_model1.orientations.df['Y']
or_z = geo_model1.orientations.df['Z']
or_dip = geo_model1.orientations.df['dip']
or_azimuth = geo_model1.orientations.df['azimuth']

surf_indexes = list(geo_model1.surface_points.df.index)
or_indexes = list(geo_model1.orientations.df.index)

In [23]:
# make a copy of the initial dataframe
or_1 = geo_model1.orientations.df.copy()

# define concentration parameter for input data
kappadict_1 = {'fault_1' : 100, 'fault_2' : 100, 'fault_3' : 100, 'fault_4' : 100,
               'fault_5' : 100,
               'UFM' : 100, 'UMM' : 100, 'LFM' : 100, 'helvetic' : 100, 'mesozoic' : 100}

# assign kappa values to the copied dataframe
for surface, kappa in kappadict_1.items():
    or_1.loc[or_1['surface'] == surface, 'kappa'] = kappa #orientations now has a new column with kappa
    
kappas = or_1['kappa']

In [24]:
#in this list, a distribution for every orientation data point is stored, we can sample from that later.
vMF_list = create_vMF_list(geo_model1, kappas, datatype = 'all')

In [25]:
#check the model extent
max_z = surf_z_1.min() 
min_z = surf_z_1.max() 

#from there downwards, uncertainty increases linearly
min_uncert = 100
max_uncert = 1000

### Simulation

In [26]:
#define for cleaner code
subsurf_x = geo_model1.surface_points.df['X'][geo_model1.surface_points.df['Z']<=500]
subsurf_y = geo_model1.surface_points.df['Y'][geo_model1.surface_points.df['Z']<=500]
subsurf_z = geo_model1.surface_points.df['Z'][geo_model1.surface_points.df['Z']<=500]

surf_x = geo_model1.surface_points.df['X'][geo_model1.surface_points.df['Z']>500]
surf_y = geo_model1.surface_points.df['Y'][geo_model1.surface_points.df['Z']>500]
surf_z = geo_model1.surface_points.df['Z'][geo_model1.surface_points.df['Z']>500]

mask = np.ones(len(geo_model1.orientations.df), dtype = bool)

#### Attempt to establish priors

In [27]:
# priors: sample from each distribution
def evaluate_sigma(iteration,surface_name, depth_points):
    
    dp_subsurf = depth_points[depth_points <= 500]
    dp_surf = depth_points[depth_points > 500]
    
    depth_uncert_subsurf = np.abs((max_uncert*dp_subsurf[surface_name].dropna())/max_z)
    
    if surface_name in ['fault_3','fault_4']:
        sigma_1 = np.random.normal(0, 2*(depth_uncert_subsurf))
        sigma_2 = np.random.normal(0, 2*(depth_uncert_subsurf))
        sigma_3 = np.random.normal(0, 3*(depth_uncert_subsurf))
        sigma_4 = np.random.uniform(-200, 200)
        sigma_5 = np.random.uniform(-200, 200)
        sigma_6 = None
        sigma_7 = None
        sigma_8 = None
        sigma_9 = None
        sigma_10 = None
        sigma_11 = None
        
    else:
        sigma_1 = None
        sigma_2 = None
        sigma_3 = None
        sigma_4 = None
        sigma_5 = None
        sigma_6 = np.random.normal(0, np.abs(depth_uncert_subsurf))
        sigma_7 = np.random.normal(0, np.abs(depth_uncert_subsurf))
        sigma_8 = np.random.normal(0, np.abs(1.5*(depth_uncert_subsurf)))
        sigma_9 = np.random.uniform(-100, 100)
        sigma_10 = np.random.uniform(-100, 100)
        sigma_11 = np.random.normal(0, 5)
    
    return [iteration,sigma_1,sigma_2,sigma_3,sigma_4,sigma_5,sigma_6,sigma_7,sigma_8,sigma_9,sigma_10,sigma_11,surface_name]

## Add uncertainty 

In [28]:
depth_points = pd.read_csv('depth_points - Copy.csv')

In [33]:
surfaces_1 = np.unique(geo_model1.surface_points.df['surface'])
voxels =100*100*50
modify_by_surface = True
modify_orientations = True
save = True

n_iterations = 1000

for iteration in tqdm(range(n_iterations)):
    #reset model to original
    geo_model1.modify_surface_points(surf_indexes, X = surf_x_1, Y = surf_y_1, Z = surf_z_1)
    geo_model1.modify_orientations(or_indexes, X = or_x, Y = or_y, Z = or_z, 
                                       dip = or_dip, azimuth = or_azimuth)
    geo_model1.update_to_interpolator()
        
    if modify_by_surface == True:
        sigma_list = []
        for surf in surfaces_1:    
            if surf in ['fault_3','fault_4']:
                sigma_list.append(evaluate_sigma(iteration,surf, depth_points))

                subsurf_x[geo_model1.surface_points.df['surface'] == surf] = subsurf_x[geo_model1.surface_points.df['surface'] == surf]  + sigma_list[-1][1]
                subsurf_y[geo_model1.surface_points.df['surface'] == surf] = subsurf_y[geo_model1.surface_points.df['surface'] == surf]  + sigma_list[-1][2]

                # add uncertainty to subsurface position z (uncertainty increases with depth)
                subsurf_z[geo_model1.surface_points.df['surface'] == surf] = subsurf_z[geo_model1.surface_points.df['surface'] == surf]  + sigma_list[-1][3]

                # add uncertainty to surface position x,y
                surf_x[geo_model1.surface_points.df['surface'] == surf] = surf_x[geo_model1.surface_points.df['surface'] == surf]  + sigma_list[-1][4]
                surf_y[geo_model1.surface_points.df['surface'] == surf] = surf_y[geo_model1.surface_points.df['surface'] == surf]  + sigma_list[-1][5]

            else:
                   
                sigma_list.append(evaluate_sigma(iteration,surf, depth_points))

                subsurf_x[geo_model1.surface_points.df['surface'] == surf] = subsurf_x[geo_model1.surface_points.df['surface'] == surf]  + sigma_list[-1][6]
                subsurf_y[geo_model1.surface_points.df['surface'] == surf] = subsurf_y[geo_model1.surface_points.df['surface'] == surf]  + sigma_list[-1][7]

                # add uncertainty to subsurface position z (uncertainty increases with depth)
                subsurf_z[geo_model1.surface_points.df['surface'] == surf] = subsurf_z[geo_model1.surface_points.df['surface'] == surf]  + sigma_list[-1][8]

                # add uncertainty to surface position x,y
                surf_x[geo_model1.surface_points.df['surface'] == surf] = surf_x[geo_model1.surface_points.df['surface'] == surf]  + sigma_list[-1][9]
                surf_y[geo_model1.surface_points.df['surface'] == surf] = surf_y[geo_model1.surface_points.df['surface'] == surf]  + sigma_list[-1][10]

                # add z uncertainty to surface data
                surf_z[geo_model1.surface_points.df['surface'] == surf] = surf_z[geo_model1.surface_points.df['surface'] == surf]  + sigma_list[-1][11]

        if modify_orientations == True:
            new_orientations = np.vstack(list(map(lambda x : x.sample(num_samples = 1, 
                                                                      direct_output = True)[0],vMF_list)))
            a = fish.vMF()
            a.add_orientation_data(new_orientations)

            geo_model1.orientations.df.loc[mask, ['G_x', 'G_y', 'G_z']] = new_orientations
            geo_model1.orientations.df.loc[mask, 'azmiuth'] = a.samples_azdip[:,0]
            geo_model1.orientations.df.loc[mask, 'dip'] = a.samples_azdip[:,1]
        
        geo_model1.update_to_interpolator()
        gp.compute_model(geo_model1)
        
    if save == True:
        np.save('blocks0/1_%04d.npy'%iteration, geo_model1.solutions.block_matrix[0,0:voxels])
        np.save('blocks1/1_%04d.npy'%iteration, geo_model1.solutions.block_matrix[1,0:voxels])
        np.save('blocks2/1_%04d.npy'%iteration, geo_model1.solutions.block_matrix[2,0:voxels])
        np.save('blocks3/1_%04d.npy'%iteration, geo_model1.solutions.block_matrix[3,0:voxels])
        np.save('blocks4/1_%04d.npy'%iteration, geo_model1.solutions.block_matrix[4,0:voxels])
        np.save('blocks5/1_%04d.npy'%iteration, geo_model1.solutions.block_matrix[5,0:voxels])
    

100%|████████████████████████████████████████████████████████████████████████████| 471/471 [15:58:28<00:00, 122.10s/it]
